In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
def scrape_nbe_faqs(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        accordion = soup.find('div', class_='elementor-accordion')
        
        if not accordion:
            print("Could not find FAQ accordion container")
            return []
            
        faq_list = []
        
        
        faq_items = accordion.find_all('div', class_='elementor-accordion-item')
        
        for item in faq_items:
            try:
                
                question_div = item.find('a', class_='elementor-accordion-title')
                question = question_div.get_text(strip=True) if question_div else ""
                
                
                answer_div = item.find('div', class_='elementor-tab-content')
                answer = ""
                
                if answer_div:
                    
                    for element in answer_div.find_all(['p', 'span']):
                        if element.name == 'p':
                            answer += element.get_text(strip=True) + "\n"
                        elif element.name == 'span' and 'relative' in element.get('class', []):
                            answer += element.get_text(strip=True) + " "
                
                if question and answer:
                    faq_list.append({
                        "instruction": "Answer the following question about the National Bank of Ethiopia",
                        "input": question,
                        "output": answer.strip(),
                        "source": url,
                        "category": "National Bank of Ethiopia FAQ"
                    })
                    
            except Exception as e:
                print(f"Error processing item: {str(e)}")
                continue
                
        return faq_list
        
    except Exception as e:
        print(f"Scraping failed: {str(e)}")
        return []


url = "https://nbe.gov.et"
faqs = scrape_nbe_faqs(url)

if faqs:
    with open('nbe_faqs.json', 'w', encoding='utf-8') as f:
        json.dump(faqs, f, indent=2, ensure_ascii=False)
    
    print(f"Successfully scraped {len(faqs)} FAQs")
    print("Sample FAQ:")
    print(json.dumps(faqs[0], indent=2, ensure_ascii=False))
else:
    print("No FAQs found.")

Successfully scraped 6 FAQs
Sample FAQ:
{
  "instruction": "Answer the following question about the National Bank of Ethiopia",
  "input": "What is the National Bank of Ethiopia?",
  "output": "TheNational Bank of Ethiopiais thecentral bank of Ethiopia. It plays a pivotal role in the country’s economic and financial system by serving as theregulator of the financial sectorand theformulator and implementer of monetary policy.\nLearn more about what we do and how we do it.\nFind out about our roles and function",
  "source": "https://nbe.gov.et",
  "category": "National Bank of Ethiopia FAQ"
}
